In [2]:
import open3d
import numpy as np
import pandas as pd
import os
import tqdm
import copy
import math

In [3]:
def preprocess_pcd(pcd, voxel_size, down_sample=True):
    if down_sample:
        pcd = open3d.voxel_down_sample(pcd, voxel_size)

    radius_normal = voxel_size * 2
    open3d.geometry.estimate_normals(pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    pcd_fpfh = open3d.registration.compute_fpfh_feature(pcd, open3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd, pcd_fpfh


def execute_global_registration(source_down, target_down, source_fpfh, target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    result = open3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, distance_threshold,
        open3d.registration.TransformationEstimationPointToPoint(False), 4, 
        [open3d.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9), open3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        open3d.registration.RANSACConvergenceCriteria(4000000, 500))
    return result


def refine_registration(source, target, voxel_size, t):
    distance_threshold = voxel_size * 1
    result = open3d.registration.registration_icp(
        source, target, distance_threshold, t,
        open3d.registration.TransformationEstimationPointToPlane(),
        open3d.registration.ICPConvergenceCriteria(max_iteration=200)
    )
    return result

def read_pcd_file(file_path, voxel_size):
    pcd = open3d.io.read_point_cloud(file_path)
    pcd = open3d.geometry.voxel_down_sample(pcd, voxel_size)
    return pcd

In [3]:
data_dir = "data/DatasetV2/Sequence/06"

pcd_files = os.listdir(data_dir)
pcd_files = sorted(pcd_files, key=lambda f: int(f.split(".")[0].split("_")[1]))

In [25]:
source = open3d.io.read_point_cloud(os.path.join(data_dir, pcd_files[0]))
target = open3d.io.read_point_cloud(os.path.join(data_dir, pcd_files[1]))

source, source_fpfh = preprocess_pcd(source, 0.05)
target, target_fpfh = preprocess_pcd(target, 0.05)

source.paint_uniform_color([0.906, 0.298, 0.236])
target.paint_uniform_color([0.204, 0.596, 0.859])

ransac_result = execute_global_registration(source, target, source_fpfh, target_fpfh, 0.05)

source.transform(ransac_result.transformation)

open3d.draw_geometries([source, target])

In [30]:
pcds = []
n_samples = 3

for i in range(len(pcd_files) // n_samples):

    source = open3d.io.read_point_cloud(os.path.join(data_dir, pcd_files[n_samples * i]))

    for j in range(n_samples):
        target = open3d.io.read_point_cloud(os.path.join(data_dir, pcd_files[n_samples * i + j + 1]))

        source, source_fpfh = preprocess_pcd(source, 0.05)
        target, target_fpfh = preprocess_pcd(target, 0.05)

        source.paint_uniform_color([0.906, 0.298, 0.236])
        target.paint_uniform_color([0.204, 0.596, 0.859])

        ransac_result = execute_global_registration(source, target, source_fpfh, target_fpfh, 0.05)
        
        icp_result = refine_registration(source, target, 0.05, ransac_result.transformation)

        source.transform(icp_result.transformation)
        
        source = source + target
    
    source.paint_uniform_color([0.906, 0.298, 0.236])
    # open3d.draw_geometries([source])
    
    pcds.append(source)

In [32]:
for i in range(len(pcds) - 1):
    source, source_fpfh = preprocess_pcd(pcds[i], 0.05)
    target, target_fpfh = preprocess_pcd(pcds[i + 1], 0.05)

    source.paint_uniform_color([0.906, 0.298, 0.236])
    target.paint_uniform_color([0.204, 0.596, 0.859])

    ransac_result = execute_global_registration(source, target, source_fpfh, target_fpfh, 0.05)
    
    icp_result = refine_registration(source, target, 0.05, ransac_result.transformation)

    source.transform(icp_result.transformation)
    
    open3d.draw_geometries([source, target])

In [ ]:
output_dir = data_dir.replace("Sequence", "SequenceCombined")

if not os.path.exists(output_dir): os.mkdir(output_dir)

for i, pcd in enumerate(pcds):
    open3d.io.write_point_cloud(os.path.join(output_dir, f"seq_{data_dir[-2:]}_{i}.pcd"), pcd)

In [4]:
def make_pcd(points, color=None):
    c = [1, 0, 0] if color is None else color
    pcd = open3d.geometry.PointCloud()
    pcd.points = open3d.utility.Vector3dVector(points)
    pcd.colors = open3d.utility.Vector3dVector([c for i in range(len(points))])
    return pcd

def get_limits(pcd):
    pcd_points = np.asarray(pcd.points)

    x_min, y_min, z_min = np.min(pcd_points, axis=0)
    x_max, y_max, z_max = np.max(pcd_points, axis=0)

    return x_min, x_max, y_min, y_max, z_min, z_max

def get_grid(pcd, cell_size):
    x_min, x_max, y_min, y_max, z_min, z_max = get_limits(pcd)
    y_val = np.mean([y_min, y_max])

    points = []
    x_n = int((x_max - x_min) // cell_size)
    z_n = int((z_max - z_min) // cell_size)
    for i in range(z_n):
        z0 = float(z_min + cell_size * (i + 1))
        for j in range(x_n):
            x0 = float(x_min + cell_size * (j + 1))
            points.append([x0, y_val, z0])

    return points

def get_features(feature_file):
    data = np.load(feature_file)
    scores = data["scores"]
    features = open3d.registration.Feature()
    features.data = data["features"].T
    keypts = open3d.geometry.PointCloud()
    keypts.points = open3d.utility.Vector3dVector(data["keypts"])
    return keypts, features, scores

def filter_indices(points, p, cell_size):
    px_min = p[0] - cell_size
    px_max = p[0] + cell_size
    pz_min = p[2] - cell_size
    pz_max = p[2] + cell_size
    xf = np.logical_and(points[:, 0] > px_min, points[:, 0] < px_max)
    zf = np.logical_and(points[:, 2] > pz_min, points[:, 2] < pz_max)
    return np.logical_and(xf, zf)

def get_cell_features(feature_file, p, cell_size):
    data = np.load(feature_file)
    scores = data["scores"]

    f = filter_indices(data["keypts"], p, cell_size)
    f = np.where(f)[0]

    features = open3d.registration.Feature()
    features.data = data["features"][f].T

    keypts = open3d.geometry.PointCloud()
    keypts.points = open3d.utility.Vector3dVector(data["keypts"][f])

    return keypts, features, scores

def registration(src_keypts, tgt_keypts, src_desc, tgt_desc, distance_threshold):
    result = open3d.registration_ransac_based_on_feature_matching(
        src_keypts, tgt_keypts, src_desc, tgt_desc,
        distance_threshold,
        open3d.TransformationEstimationPointToPoint(False), 3,
        [open3d.CorrespondenceCheckerBasedOnEdgeLength(0.9), open3d.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        open3d.RANSACConvergenceCriteria(40000000, 500))
    return result

def print_registration_result(src_keypts, tgt_keypts, result_ransac, end="\n"):
    print(f"Keypts: [{len(src_keypts.points)}, {len(tgt_keypts.points)}]", end="\t")
    print(f"No of matches: {len(result_ransac.correspondence_set)}", end="\t")
    print(f"Inlier RMSE: {result_ransac.inlier_rmse:.4f}", end=end)
    
def create_camera(camera_width, color):
    points = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1], [1, 0, 1], [0, 1, 1], [1, 1, 1]])
    points = points * camera_width
    points = points - camera_width / 2
    lines = [[0, 1], [0, 2], [1, 3], [2, 3], [4, 5], [4, 6], [5, 7], [6, 7], [0, 4], [1, 5], [2, 6], [3, 7]]
    colors = [color for i in range(len(lines))]
    line_set = open3d.geometry.LineSet()
    line_set.points = open3d.utility.Vector3dVector(points)
    line_set.lines = open3d.utility.Vector2iVector(lines)
    line_set.colors = open3d.utility.Vector3dVector(colors)
    return line_set

def rgb(r, g, b):
    return r / 255.0, g / 255.0, b / 255.0

In [43]:
cell_size = 3
sequence_id = 6

sequence_dir = "data/DatasetV2/SequenceCombined/{:02d}".format(sequence_id)

tgt_pcd_file = "data/DatasetV2/Primary/06/lidar_1637299401488642900.pcd"
tgt_file_name = tgt_pcd_file.split('/')[-1]
tgt_feature_file = f"data/FeaturesV8/0.05/{tgt_file_name.replace('pcd', 'npz')}"

pcd = open3d.read_point_cloud(tgt_pcd_file)
pcd = open3d.voxel_down_sample(pcd, 0.025)
pcd.paint_uniform_color(rgb(149, 165, 166))

grid_points = get_grid(pcd, cell_size)
grid = make_pcd(grid_points)


for seq_file_name in os.listdir(sequence_dir):
    src_feature_file = os.path.join("data/FeaturesV8/0.05/", seq_file_name.replace("pcd", "npz"))
    best_score = 0
    best_p = None
    t = None

    for p in tqdm.tqdm(grid_points):
        src_keypts, src_features, src_scores = get_features(src_feature_file)
        tgt_keypts, tgt_features, tgt_scores = get_cell_features(tgt_feature_file, p, cell_size)

        if len(tgt_keypts.points) < 2000:
            continue

        src_keypts.paint_uniform_color([1, 0.706, 0])
        tgt_keypts.paint_uniform_color([0, 0.651, 0.929])

        result_ransac = registration(src_keypts, tgt_keypts, src_features, tgt_features, 0.05)
        
        # print_registration_result(src_keypts, tgt_keypts, result_ransac, end="\n")

        if best_score < len(result_ransac.correspondence_set):
            best_score = len(result_ransac.correspondence_set)
            best_p = p
            t = result_ransac.transformation
            
    if best_p is not None:
        src_keypts, src_features, src_scores = get_features(src_feature_file)

        src_keypts.paint_uniform_color([1, 0.706, 0])

        src_camera = create_camera(camera_width=0.25, color=[1, 0, 0])

        result = open3d.registration.registration_icp(
            src_keypts, pcd, 0.05, t, open3d.registration.TransformationEstimationPointToPoint()
        )

        src_keypts.transform(result.transformation)
        src_camera.transform(result.transformation)
        open3d.visualization.draw_geometries([src_keypts, pcd, src_camera, grid])


In [15]:
def rotate_transformation_matrix(t, rx, ry, rz):
    # Convert degrees to radians
    rx, ry, rz = np.radians(rx), np.radians(ry), np.radians(rz)

    RX = np.array([
        [1, 0, 0, 0],
        [0, np.cos(rx), -np.sin(rx), 0],
        [0, np.sin(rx), np.cos(rx), 0],
        [0, 0, 0, 1]
    ])

    RY = np.array([
        [np.cos(ry), 0, np.sin(ry), 0],
        [0, 1, 0, 0],
        [-np.sin(ry), 0, np.cos(ry), 0],
        [0, 0, 0, 1]
    ])

    RZ = np.array([
        [np.cos(rz), -np.sin(rz), 0, 0],
        [np.sin(rz), np.cos(rz), 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

    return np.dot(np.dot(np.dot(t, RZ), RY), RX)


def get_transformation_matrix(translation, rotation):
    t = np.identity(4)
    t = rotate_transformation_matrix(t, *rotation)
    t[:, 3] = translation
    return t


def rotation(t):
    rx = math.degrees(math.atan2(t[2][1], t[2][2]))
    ry = math.degrees(math.atan2(-t[2][0], math.sqrt(t[2][1] ** 2 + t[2][2] ** 2)))
    rz = math.degrees(math.atan2(t[1][0], t[0][0]))
    return np.array([rx, ry, rz])


def translation(t):
    return np.array([t[0][3], t[1][3], t[2][3]])


def inv_transform(T):
    T_inv = np.identity(4)
    T_inv[:3, :3] = T[:3, :3].T
    T_inv[:3, 3] = -np.dot(T[:3, :3].T, T[:3, 3])
    return T_inv

def calc_error(T1, T2):
    e1 = np.sum(np.abs(rotation(T1) - rotation(T2)))
    e2 = np.sum(np.abs(translation(T1) - translation(T2)))
    return e1, e2

def check(T1, T2, max_r=5, max_t=1):
    er, et = calc_error(T1, T2)
    print(f"Rotation: {er}, Translation: {et}")
    return er < max_r and et < max_t

In [15]:
data_dir = "data/DatasetV2/Sequence/09"
output_dir = data_dir.replace("Sequence", "SequenceCombined")

pcd_files = os.listdir(data_dir)
pcd_files = np.array(sorted(pcd_files, key=lambda f: int(f.split(".")[0].split("_")[1])))

# fps = int(1 / np.mean([(int(pcd_files[i + 1][6:-4]) - int(pcd_files[i][6:-4])) / 1e9 for i in range(100)]))

inds = np.arange(0, len(pcd_files), 3)
pcd_files = pcd_files[inds]

if not os.path.exists(output_dir): os.mkdir(output_dir)

for i, pcd_file in enumerate(pcd_files):
    pcd = open3d.io.read_point_cloud(os.path.join(data_dir, pcd_file))
    pcd = open3d.voxel_down_sample(pcd, 0.05)
    open3d.io.write_point_cloud(os.path.join(output_dir, f"seq_{data_dir[-2:]}_{i}.pcd"), pcd)

In [94]:
def show_registration(pcd1, pcd2, transformation):
    source = copy.deepcopy(pcd1)
    target = copy.deepcopy(pcd2)
    
    source.paint_uniform_color([0.906, 0.298, 0.236])
    target.paint_uniform_color([0.204, 0.596, 0.859])
    
    # icp_result = refine_registration(source, target, 0.05, result.transformation)
    source.transform(transformation)
    
    open3d.draw_geometries([source, target])
        

In [25]:
sequence_dir = "data/DatasetV2/SequenceCombined/06"
sequence_files = os.listdir(sequence_dir)

sequence_files = sorted(sequence_files, key=lambda x: int(x.split("_")[2].split(".")[0]))

In [7]:
local_t = []

for i in range(10):
    # # FCGF Registration
    # print("Registration between frame {} and {}".format(sequence_files[5 * i], sequence_files[5 * (i + 1)]))
    
    src_feature_file = os.path.join("data/FeaturesV8/0.05/", sequence_files[5 * i].replace("pcd", "npz"))
    tgt_feature_file = os.path.join("data/FeaturesV8/0.05/", sequence_files[5 * (i + 1)].replace("pcd", "npz"))

    src_keypts, src_features, src_scores = get_features(src_feature_file)
    tgt_keypts, tgt_features, tgt_scores = get_features(tgt_feature_file)
    
    # result_ransac = registration(src_keypts, tgt_keypts, src_features, tgt_features, 0.05)
    
    # src_keypts.paint_uniform_color([0.906, 0.298, 0.236])
    # tgt_keypts.paint_uniform_color([0.204, 0.596, 0.859])
    
    # src_keypts.transform(ransac_result.transformation)
    # open3d.visualization.draw_geometries([src_keypts, tgt_keypts])
    
    # ICP-based registration

    # source = open3d.io.read_point_cloud(os.path.join(sequence_dir, sequence_files[5 * i]))
    # target = open3d.io.read_point_cloud(os.path.join(sequence_dir, sequence_files[5 * (i + 1)]))
    
    source, source_fpfh = preprocess_pcd(src_keypts, 0.05, down_sample=False)
    target, target_fpfh = preprocess_pcd(tgt_keypts, 0.05, down_sample=False)

    source.paint_uniform_color([0.906, 0.298, 0.236])
    target.paint_uniform_color([0.204, 0.596, 0.859])

    ransac_result = execute_global_registration(source, target, source_fpfh, target_fpfh, 0.05)
    icp_result = refine_registration(source, target, 0.05, ransac_result.transformation)

    source.transform(icp_result.transformation)
    
    local_t.append(icp_result.transformation)
    
    open3d.visualization.draw_geometries([source, target])

In [39]:
local_pcds = []

trans = np.identity(4)

for i in range(10):
    source = open3d.io.read_point_cloud(os.path.join(sequence_dir, sequence_files[5 * i]))
    source, source_fpfh = preprocess_pcd(source, 0.05)
    
    t = inv_transform(local_t[i - 1]) if i > 0 else np.identity(4)
    trans = np.dot(trans, t)
    
    source.transform(trans) 
    local_pcds.append(source)
    
open3d.visualization.draw_geometries(local_pcds)  

[0.0833219661984046, 0.827247118480384, 0.8759001904171172]

In [8]:
cell_size = 3
sequence_id = 7
global_t = []

sequence_dir = "data/DatasetV2/SequenceCombined/{:02d}".format(sequence_id)

sequence_files = os.listdir(sequence_dir)
sequence_files = sorted(sequence_files, key=lambda x: int(x.split("_")[2].split(".")[0]))

tgt_pcd_file = "data/DatasetV2/Primary/07/lidar_1637299421190934300.pcd"
tgt_file_name = tgt_pcd_file.split('/')[-1]
tgt_feature_file = f"data/FeaturesV8/0.05/{tgt_file_name.replace('pcd', 'npz')}"

pcd = open3d.read_point_cloud(tgt_pcd_file)
pcd = open3d.voxel_down_sample(pcd, 0.025)
pcd.paint_uniform_color(rgb(149, 165, 166))

grid_points = get_grid(pcd, cell_size)
grid = make_pcd(grid_points)


for i in range(10):
    src_feature_file = os.path.join("data/FeaturesV8/0.05/", sequence_files[5 * i].replace("pcd", "npz"))
    best_score = 0
    best_p = None
    t = None

    for p in tqdm.tqdm(grid_points):
        src_keypts, src_features, src_scores = get_features(src_feature_file)
        tgt_keypts, tgt_features, tgt_scores = get_cell_features(tgt_feature_file, p, cell_size)

        if len(tgt_keypts.points) < 2000:
            continue

        src_keypts.paint_uniform_color([1, 0.706, 0])
        tgt_keypts.paint_uniform_color([0, 0.651, 0.929])

        result_ransac = registration(src_keypts, tgt_keypts, src_features, tgt_features, 0.05)
        
        # print_registration_result(src_keypts, tgt_keypts, result_ransac, end="\n")

        if best_score < len(result_ransac.correspondence_set):
            best_score = len(result_ransac.correspondence_set)
            best_p = p
            t = result_ransac.transformation
            
    if best_p is not None:
        src_keypts, src_features, src_scores = get_features(src_feature_file)

        src_keypts.paint_uniform_color([1, 0.706, 0])

        src_camera = create_camera(camera_width=0.25, color=[1, 0, 0])

        # result = open3d.registration.registration_icp(
        #     src_keypts, pcd, 0.05, t, open3d.registration.TransformationEstimationPointToPoint()
        # )
        
        global_t.append(t)

        src_keypts.transform(t)
        src_camera.transform(t)
        # open3d.visualization.draw_geometries([src_keypts, pcd, src_camera, grid])

100%|██████████| 35/35 [00:22<00:00,  1.53it/s]


In [44]:
for i in range(1, 10):
    src_feature_file = os.path.join("data/FeaturesV8/0.05/", sequence_files[5 * i].replace("pcd", "npz"))
    
    src_keypts, src_features, src_scores = get_features(src_feature_file)
    
    it = np.dot(global_t[i - 1], inv_transform(local_t[i - 1]))
    
    p0 = copy.deepcopy(src_keypts)
    
    # src_keypts.transform(inv_transform(local_t[i - 1]))
    # src_keypts.transform(global_t[i - 1])
    src_keypts.transform(it)
    p0.transform(global_t[i])
    
    print(rotation(it), rotation(global_t[i]))
    
    src_keypts.paint_uniform_color(rgb(231, 76, 60)) # red
    p0.paint_uniform_color(rgb(39, 174, 96)) # green
    
    open3d.draw_geometries([p0, src_keypts, pcd])

[ -6.92709024   1.39659001 177.48863044] [ -7.06672605   1.33073048 178.88761132]
[ -6.29956817   0.36831358 178.81855173] [ 93.20236538  88.36549997 -78.57371473]
[ 89.92386435  87.66055571 -80.53336256] [  -7.33357247    0.50009757 -176.89537222]
[  -7.86924584   -4.08684583 -177.8227317 ] [  -8.30191064   -3.97466792 -178.75297752]
[  -7.94502559  -20.01569581 -179.47322949] [  -8.26357792  -20.32692678 -179.57861711]
[  -8.71054551  -47.37898763 -179.93603589] [ -8.63097901 -47.67407089 179.89760495]
[ -6.74238308 -72.08554168 178.36819178] [ -5.71346938 -71.84837184 177.08917203]
[ -6.42162064 -80.97548917 176.50903115] [ -8.30181237 -80.94091182 178.35429058]
[ -4.40203684 -85.52788217 176.93979516] [ -5.11713494 -85.24148174 177.17979621]


In [106]:
for i in range(1, 10):
    p = np.load(os.path.join("data/FeaturesV8/0.05/", sequence_files[5 * i].replace("pcd", "npz")))["keypts"]
    
    p0 = make_pcd(p)
    p1 = copy.deepcopy(p0)
    
    p1.transform(inv_transform(local_t[i - 1]))
    
    p2 = copy.deepcopy(p1)
    p2.transform(global_t[i - 1])
    
    p0.paint_uniform_color(rgb(39, 174, 96)) # green - origin
    p1.paint_uniform_color(rgb(41, 128, 185)) # blue - previous global
    p2.paint_uniform_color(rgb(231, 76, 60)) # red - local
    
    open3d.draw_geometries([p0, p1, p2, pcd])

In [9]:
def validate(T1, T2, T3, t1, t2, max_dist, max_rot):
    c1 = check(T3, np.dot(T2, inv_transform(t2)), max_t=max_dist, max_r=max_rot)
    c2 = check(T3, np.dot(np.dot(T1, inv_transform(t1)), inv_transform(t2)), max_t=max_dist, max_r=max_rot)
    c3 = check(T2, np.dot(T1, inv_transform(t1)), max_t=max_dist, max_r=max_rot)

    print(f"Check 1: {c1}, Check 2: {c2}, Check 3: {c3}")
    
    # If two checks are true, the combination is wrong
    if (c1 + c2 + c3) == 2:
        raise Exception("Invalid combination")

    # If two checks are true, the combination is wrong
    if (c1 + c2 + c3) == 0:
        raise Exception("Invalid transformations")

    # If all the checks are valid, there is no need of correction
    if (c1 + c2 + c3) == 3:
        print(":: No need of correction.")
        return T1, T2, T3
    

    # If two checks are wrong, only one transformation needs correction
    if c1:
        print(":: Correcting Previous Transformation")
        T1 = np.dot(T2, t1)
    elif c2:
        print(":: Correcting Current Transformation")
        T2 = np.dot(T1, inv_transform(t1))
    else:
        print(":: Correcting Future Transformation")
        T3 = np.dot(T2, inv_transform(t2))

    return T1, T2, T3

In [16]:
from scipy.spatial.transform import Rotation as R


def rotation(transformation):
    rm = transformation[:3, :3].tolist()
    return R.from_matrix(rm).as_euler('xzy', degrees=True)

In [18]:
calc_error(np.dot(global_t[0], inv_transform(local_t[0])), global_t[1])

(4.7254846060861695, 0.2574965017294122)

In [26]:
dump = np.load("dump.npz")

local_t = dump["local_t"]
global_t = dump["global_t"]

In [27]:
global_tc = copy.deepcopy(global_t)

for i in range(1, 9):
    print("============================{}-{}-{}============================".format(i - 1, i, i + 1))
    try:
        print(rotation(global_tc[i - 1]) + rotation(inv_transform(local_t[i - 1])), rotation(global_tc[i]))
        global_tc[i - 1], global_tc[i], global_tc[i + 1] = validate(global_tc[i - 1], global_tc[i], global_tc[i + 1], local_t[i - 1], local_t[i], 1, 10)
    except Exception as e:
        print(e)

============================0-1-2============================
[173.13961108   2.84619417 178.35309135] [172.9591108    1.11208862 178.66901877]
Rotation: 94.67549470196907, Translation: 2.2641716973663906
Rotation: 96.16189317741807, Translation: 2.078281210267985
Rotation: 1.6405323103986524, Translation: 0.18605985176954754
Check 1: False, Check 2: False, Check 3: True
:: Correcting Future Transformation
============================1-2-3============================
[173.7257484    0.92515893 177.72240649] [173.70802754   1.18142385 179.63160811]
Rotation: 3.924802006508413, Translation: 0.11032768148268601
Rotation: 3.924802006508413, Translation: 0.11032768148268601
Rotation: 0.0, Translation: 0.0
Check 1: True, Check 2: True, Check 3: True
:: Correcting Previous Transformation
============================2-3-4============================
[172.38828354   1.88837007 179.59105607] [172.63930305  -3.10450941 179.49916739]
Rotation: 1.5433466672882812, Translation: 0.08888939285723807
R

In [30]:
for i in range(1, 10):
    src_feature_file = os.path.join("data/FeaturesV8/0.05/", sequence_files[5 * i].replace("pcd", "npz"))
    
    src_keypts, src_features, src_scores = get_features(src_feature_file)
    
    it = np.dot(global_t[i - 1], inv_transform(local_t[i - 1]))
    
    p0 = copy.deepcopy(src_keypts)
    p1 = copy.deepcopy(src_keypts)
    p2 = copy.deepcopy(src_keypts)
    
    p0.transform(global_t[i])
    p1.transform(it)
    p2.transform(global_tc[i])
    
    
    p0.paint_uniform_color(rgb(231, 76, 60)) # red - local
    p1.paint_uniform_color(rgb(39, 174, 96)) # green - origin
    p2.paint_uniform_color(rgb(41, 128, 185)) # blue - correction
    
    open3d.draw_geometries([p0, p2, pcd])

In [31]:
trajectory = [global_tc[i][:3, 3].tolist() for i in range(10)]
lines = [[i, i + 1] for i in range(len(trajectory) - 1)]

colors = [[1, 0, 0] for i in range(len(lines))]
line_set = open3d.geometry.LineSet()
line_set.points = open3d.utility.Vector3dVector(trajectory)
line_set.lines = open3d.utility.Vector2iVector(lines)
line_set.colors = open3d.utility.Vector3dVector(colors)

open3d.visualization.draw_geometries([pcd, line_set])

In [48]:
def asd():
    c1 = True
    c2 = True
    c3 = True
    # If two checks are true, the combination is wrong
    if (c1 + c2 + c3) == 2:
        raise Exception("Invalid combination")

    # If two checks are true, the combination is wrong
    if (c1 + c2 + c3) == 0:
        raise Exception("Invalid transformations")

    # If all the checks are valid, there is no need of correction
    if (c1 + c2 + c3) == 3:
        print(":: No need of correction.")
        return None
    
    # If two checks are wrong, only one transformation needs correction
    if c1:
        print(":: Correcting Previous Transformation")
    elif c2:
        print(":: Correcting Current Transformation")
    else:
        print(":: Correcting Future Transformation")

    return None

In [49]:
asd()

:: No need of correction.
